In [ ]:
import boto3
import sagemaker
import pandas as pd
import json

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
%store -r autopilot_endpoint_name

In [ ]:
if not autopilot_endpoint_name:
    print('****************************************************************************************')
    print('**************** PLEASE WAIT FOR THE PREVIOUS NOTEBOOK TO FINISH ***********************')
    print('******************** OR THIS NOTEBOOK WILL NOT RUN PROPERLY ****************************')
    print('****************************************************************************************')

In [ ]:
print(autopilot_endpoint_name)

# Wait for the Model to Deploy
This may take 5-10 mins.  Please be patient.

In [ ]:
sm.get_waiter('endpoint_in_service').wait(EndpointName=autopilot_endpoint_name)

In [ ]:
resp = sm.describe_endpoint(EndpointName=autopilot_endpoint_name)
status = resp['EndpointStatus']

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

# Test Our Model with Some Example Reviews
Let's do some ad-hoc predictions on our model.

In [ ]:
sm_runtime = boto3.client('sagemaker-runtime')

In [ ]:
csv_line_predict_positive = """I loved it!"""

response = sm_runtime.invoke_endpoint(EndpointName=autopilot_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_positive)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

In [ ]:
csv_line_predict_meh = """It's OK."""

response = sm_runtime.invoke_endpoint(EndpointName=autopilot_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_meh)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

In [ ]:
csv_line_predict_negative = """The worst product ever."""

response = sm_runtime.invoke_endpoint(EndpointName=autopilot_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_negative)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

# Summary
We used Autopilot to automatically find the best model, hyper-parameters, and feature-engineering scripts for our dataset.  

Autopilot uses a transparent approach to generate re-usable exploration Jupyter Notebooks and transformation Python scripts to continue to train and deploy our model on new data - well after this initial interaction with the Autopilot service.

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();